---
# Tutorial Task: Sentiment-Based Quant Strategy
---

In this tutorial, you will design and implement a **quantitative trading strategy** that makes trading decisions based on **news sentiment** for a selected group of 5 stocks.

## Objective

Each day, over a 1-week historical period:
- **Buy** (go long) the stock with the **most positive** news sentiment
- **Sell** (go short) the stock with the **most negative** news sentiment

You will:
1. Select your own **universe of 5 stocks** (e.g., AAPL, MSFT, GOOGL, AMZN, TSLA).
2. Use the provided News API to retrieve articles for each stock.
3. Calculate the **average sentiment** for each stock daily using tools like `TextBlob`.
4. Track your portfolio’s value over time by simulating simple returns (real or mock).
5. Plot your portfolio value and analyze strategy performance.

## Constraints

- The News API is limited to **100 requests per day**. Avoid excessive API calls while developing by storing data or limiting requests.
- You only trade **one long and one short position** per day.
- Keep it simple: assume equal capital allocation to long and short trades.

## Deliverables

- A working Jupyter notebook implementing the above strategy
- Comments and clear structure for readability
- A final plot showing portfolio value over the 1-week period


## Step 1:
Import relevant libraries

## Step 2:
Configure API

## Step 3:
Create a function to get news and calculate the sentiment

## Step 4:
Simulate Trading

## Step 5:
Display results in ```matplotlib```